In [1]:
import numpy as np
import shapely as sh
from uuid import uuid4
from sys import modules
from time import perf_counter
from rebound import Simulation
import matplotlib.colors as colors
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit
from multiprocessing.pool import Pool

def globalize(func):
    def result(*args, **kwargs):
        return func(*args, **kwargs)
    result.__name__ = result.__qualname__ = uuid4().hex
    setattr(modules[result.__module__], result.__name__, result)
    return result

def timeit(f, *args, **kwargs):
    start = perf_counter()
    f(*args, **kwargs)
    end = perf_counter()
    time = end-start
    print(f"Elapsed time = {time}s")
    
def HW99(mu: float, e: float, a_b: float) -> float:
    return (0.464-0.380*mu-0.631*e+0.586*mu*e
            +0.150*e**2-0.198*mu*e**2)*a_b

In [35]:
def init(M_A: float, M_B: float, a_bin: float, a_p: float, 
         inc_bin: float, Omega_bin:float, inc_p: float, Omega_p: float,
         e_B: float = 0, e_p: float = 0, omega: float = 0) -> Simulation:
    """Returns a Simulation object of all 3 objects in the system.
    
    Parameter
    ---------
    M_A : float
        Mass of star A (solar masses)
    M_B : float
        Mass of star B (solar masses)
    a_bin : float
        Semi-major axis of binary (AU)
    a_p : float
        Semi-major axis of planet (AU)
    inc_bin : float
        Inclination of binary (rad)
    Omega_bin : float
        Longitude of ascending node of binary (rad)
    inc_p : float
        Inclination of planet (rad)
    Omega_p : float
        Longitude of ascending node of planet (rad)
    e_B : float (default 0.0)
        Orbital eccentricity of binary
    e_p : float (default 0.0)
        Orbital eccentricity of planet
    omega : float (default 0.0)
        Argument of pericenter (rad)
    """
    sim = Simulation()
    sim.add(m=M_A)
    sim.add(m=M_B, a=a_bin, inc=inc_bin, 
            Omega=Omega_bin, e=e_B, omega=omega)
    sim.add(a=a_p, inc=inc_p, Omega=Omega_p, e=e_p)
    sim.move_to_com()
    return sim

def integrate(sim: Simulation, N_orbits: float = 1) -> tuple[np.array]:
    """Returns arrays of the x and y coordinates of the particles 
    in the Simulation object after being integrated over 'Norbits' 
    planet orbits.
    
    Parameter
    ---------
    sim : Simulation
        Simulation object
    Norbits : float (default 1.0)
        Number of planet orbits to plot
    """
    N_STEPS = 1000
    P = sim.calculate_orbits()[1].P
    T = np.linspace(0, N_orbits*P, N_STEPS)
    x = np.zeros((sim.N, N_STEPS))
    y = np.zeros_like(x)
    for i, t in enumerate(T):
        sim.integrate(t)
        for j in range(sim.N):
            x[j,i] = sim.particles[j].x
            y[j,i] = sim.particles[j].y
    return x, y

def plot(sim: Simulation, x: np.array, y: np.array) -> None:
    """Returns None and plots the orbits.

    Parameter
    ---------
    sim : Simulation
        Simulation object
    x: np.array
        x-coordinates of the bodies in the system
    y: np.array
        y-coordinates of the bodies in the system
    """

    fig , ax = plt.subplots()
    for n in range(sim.N):
        ax.plot(x[n], y[n])
    ax.set_title("View of the System along the z-axis")
    ax.set_xlabel("x-coordinate")
    ax.set_ylabel("y-coordinate")
    plt.grid(True)
    
def plot_system(M_A: float, M_B: float, a: float, r_a: float, 
                inc: float, Omega:float = 0, e: float = 0, 
                omega: float = 0, N: float = 1) ->  None:
    """Returns None and plots the orbits.
    
    Parameter
    ---------
    M_A : float
        Mass of star A (solar masses)
    M_B : float
        Mass of star B (solar masses)
    a : float
        Semi-major axis of binary (AU)
    r_a : float
        Ratio of the planet's semi-major axis 
        and the binary's semi-major axis
    inc : float
        Inclination of binary (rad)
    Omega : float (default 0.0)
        Longitude of ascending node of binary (rad)
    e : float (default 0.0)
        Orbital eccentricity of binary
    omega : float (default 0.0)
        Argument of pericenter (rad)
    N : float (default 1.0)
        Orbital eccentricity of planet
    """
    sim = init(M_A, M_B, a, r_a*a, inc, 
               Omega, inc, Omega, e, omega=omega)
    x, y = integrate(sim, N)
    plot(sim, x, y)
    
def check_transit(R_A: float, R_B: float, R_p: float, 
                  x: np.array, y: np.array, mu: float) -> int:
    """Returns an int corresponding the number of stars 
    in the binary that the planet transits.

    Parameter
    ---------
    R_A : float
        Radius of star A (AU)
    R_B : float
        Radius of star B (AU)
    R_p : float
        Radius of planet (AU)
    x : np.array
        x-coordinates of the bodies in the system
    y : np.array
        y-coordinates of the bodies in the system
    mu : float
        Mass ratio of the binary
    """
    coords = np.transpose(
        np.array([[x[i], y[i]] for i in range(x.shape[0])]), axes=(0,2,1))
    ringA = sh.LinearRing(coords[0])
    ringB = sh.LinearRing(coords[1])
    ringP = sh.LinearRing(coords[2])
    return 2 * int(sh.dwithin(ringA, ringP, R_A+R_p) # temporary fix
                   or sh.dwithin(ringB, ringP, R_B+R_p))
    
def transit_plot(r_A: float, r_B: float, r_p: float, I_range: tuple, 
                 a_r_range: tuple, a: float = 1, M: float = 1, mu: float = 1, 
                 Omega: float = 0, e: float = 0, omega: float = 0) -> None:
    """Returns None and creates a plot of the transit function over 
    the domain of the inclination range and the semi-major axis ratio range.
    
    Parameter
    ---------
    r_A : float
        Radius of star A (AU)
    r_B : float
        Radius of star B (AU)
    r_p : float
        Radius of planet (AU)
    I_range : tuple[float]
        Inclination range of system (rad)
    a_r_range : tuple[float]
        Ratio of the planet's semi-major axis 
        and the binary's semi-major axis
    a : float (default 1.0)
        Semi-major axis of binary (AU)
    M : float (default 1.0)
        Mass of star A (solar masses)
    mu : float (default 1.0)
        Mass ratio between stars
    Omega : float (default 0.0)
        Longitude of ascending node of system (rad)
    e : float (default 0.0)
        Orbital eccentricity of planet
    omega : float (default 0.0)
        Argument of pericenter (rad)
    """
    I = np.arange(*I_range)
    ratio = np.arange(*a_r_range)
    args = np.array(np.meshgrid(I, ratio)).T.reshape(-1,2)
    
    @globalize
    def process(i, r) -> int:
        i += 90
        a_critical = HW99(mu/(1+mu), e, a)
        if a_critical > a*r:
            return -1
        sim = init(M, mu*M, a, a*r, i*np.pi/180, Omega*np.pi/180, 
           i*np.pi/180, Omega*np.pi/180, e_B=e, omega=90*np.pi/180)
        x, y = integrate(sim, 1)
        return check_transit(r_A, r_B, r_p, x, y, mu)
        
    results = np.array([]);
    with Pool() as pool:
        for result in pool.starmap(process, args):
            results = np.append(results, result);
            
    def fit(grid: np.array, x: np.array, y: np.array, R: float) -> None:
        def curve(r, A): # need to tweak
            return A*R/r
        boundary = np.empty((0,2))
        for i, col in enumerate(grid.T):
            steps = 0
            for j, val in enumerate(col):
                if val:
                    steps += 1
            if not steps or steps==col.size:
                continue
            point = np.array([[x[i], y[steps-1]]])
            boundary = np.concatenate((boundary, point))
        _x, _y = boundary.T[0], boundary.T[1]
        popt, pcov = curve_fit(curve, _x, _y)
        plt.plot(x, curve(x, *popt), c='k')

    transits = np.array(results).reshape(I.size, ratio.size)
    fig, ax = plt.subplots()
    cmap = colors.ListedColormap(['black', 'red', 'orange', 'green'])
    boundaries = [-1.5, -0.5, 0.5, 1.5, 2.5]
    norm = colors.BoundaryNorm(boundaries, cmap.N, clip=True)
    ax.pcolormesh(ratio, I, transits, cmap=cmap, norm=norm)
    # fit(transits, ratio, I, r_A+r_B) # work in progress
    plt.ylim(I.min(), I.max())
    plt.xlabel('$a_p/a_b$')
    plt.ylabel('$I\ (^\circ)$')
    description = f'$e=${e}\n$\omega$={omega}$^\circ$'
    props = dict(boxstyle='round', facecolor='white', alpha=0.5)
    ax.text(0.05, 0.95, description, transform=ax.transAxes, fontsize=12,
         verticalalignment='top', bbox=props)
    plt.show()
    # plt.savefig(f'e{int(100*e)}omega{int(omega)}.png')
    # plt.close()

In [36]:
# Example plot
transit_plot(0.1, 0.1, 0.05, (0, 5, 0.1), (2, 5, 0.1))

e, omega = (0, 0.25, 0.5, 0.75), (0, 45, 90)
args = np.array(np.meshgrid(e, omega)).T.reshape(-1,2)
for _e, _omega in args:
    transit_plot(0.1, 0.1, 0.05, (0, 5, 0.1), (2, 5, 0.1), e=_e, omega=_omega)

import os
import cv2
d = os.listdir('./')
img_lst = np.array([f for f in d if '.png' in f]).sort().reshape(4,3).tolist()
for i, col in enumerate(img_lst):
    for j, filename in enumerate(col):
        col[j] = cv2.imread(filename)
def concat_vh(list_2d):
    return cv2.vconcat([cv2.hconcat(list_h) for list_h in list_2d])
cv2.imwrite('finalplots.png', concat_vh(img_lst))